## Part 1

In [1]:
import pandas as pd
import numpy as np
import ast
import re
from collections import defaultdict

### load business data and review data

trainSize=10000
business_train = pd.read_json('..//data//business_train.json',orient = 'records',lines = True)
review_train=pd.read_json('..//data//review_train.json',orient='records',lines=True,chunksize=trainSize)

business_train.shape

(154606, 11)

In [2]:
### first look at the data structure of business_train


business_train.categories.iloc[:5]

businessCategory=business_train.categories.map(lambda x: x.split(',') if x!=None else [])
res=defaultdict(int)
for i in businessCategory:
    for j in i:
        res[j]+=1

res=sorted(res.items(),key=lambda x:x[1],reverse=True)





### filter all the Chinese Restaurants (let's do something simple first) 

### we change the filter function to read zihanz's topic model's contents

#business_train=busines    s_train[business_train.categories.map(lambda x:"Restaurants" in x and "Chinese" in x if x else False)]
#business_train=business_train[business_train.categories.map(lambda x:"Restaurants" in x  if x else False)]



id=pd.read_csv("../data/id.txt",header=None)
id=id[0]
topic=pd.read_csv("../data/yaoshen.csv")

In [3]:
business_train=business_train.set_index('business_id').join(topic.set_index('business_id')[['fastfood','bars','japan','asian']],how='right')

In [4]:
business_train=business_train[business_train.index.isin(id)]

In [5]:
business_train['business_id'] = business_train.index

In [6]:
business_train=business_train.reset_index(drop=True)

In [7]:
business_train.shape

(37775, 15)

In [20]:
for i in range(4):
    if i==0:   
        label="fastfood"
        business_train_subset=business_train[business_train['fastfood']==1]
    elif i==1:
        label="bars"
        business_train_subset=business_train[business_train['bars']==1]
    elif i==2:
        label="japan"
        business_train_subset=business_train[business_train['japan']==1]
    else:
        label="asian"
        business_train_subset=business_train[business_train['asian']==1]
    
    
    #business_train_subset=business_train_subset[business_train_subset.categories.map(lambda x:"Sushi Bars" in x and "Japanese" in x  if x else False) & business_train_subset.city.map(lambda x: x=="Las Vegas")]

    ### preserve business_id, stars columns and save it to local json files

    # count=2
    # res=next(review_train).iloc[:,[0,2]]
    # for reviewTrainChunk in review_train:
    #     print("processing {} th chunk ...".format(count))
    #     res=res.append(reviewTrainChunk.iloc[:,[0,2]])
    #     count+=1
    # print("finished")

    # res.shape
    # res.columns

    # res=res.groupby('business_id',as_index=False).mean()

    # res.to_json("average_star_by_shops.json",orient='records')

    shop_stars=pd.read_json("average_star_by_shops.json",orient="records")

    ### Extract nested json data from business attribute data

    def nestedValue(x:str):
        """judge if a string contains nested json information"""
        return re.match(u'{.*}',x)

    ### first extract the column names from nested json
    colnames=[]
    for attribute in business_train_subset.attributes:
        if attribute==None:
            continue
        for key,value in attribute.items():
            if not nestedValue(value):
                colnames.append(key)
            else:
                for nestKey,nestValue in ast.literal_eval(value).items():
                    colnames.append(nestKey)
    colnames=set(colnames)

    ### second extract the value from nested json file
    res=defaultdict(list)
    count=0
    for attribute in business_train_subset.attributes:


        for i in colnames:
            res[i].append(None)
        if attribute==None:
            count+=1
            continue
        for key,value in attribute.items():
            if not nestedValue(value):
                res[key][count]=value
            else:
                for nestKey,nestValue in ast.literal_eval(value).items():
                    res[nestKey][count]=nestValue


        count+=1 

    attributeTrain=pd.DataFrame(res)

    attributeTrain.shape

    attributeTrain['business_id']=business_train_subset.business_id.tolist()

    ### join business data and review rating on business_id

    attributeTrain=attributeTrain.join(shop_stars.set_index('business_id'),on="business_id",how='left')

    ### data cleaning, drop columns with many NAs, convert string to category 

    #attributeTrainReduced=attributeTrain.dropna(thresh=len(attributeTrain)*0.8, axis=1)
    attributeTrainReduced=attributeTrain
    #attributeTrainReduced=attributeTrainReduced.drop("business_id",axis=1);





    attributeTrainReduced.fillna(value=pd.np.nan, inplace=True)



    for i in range(attributeTrainReduced.shape[0]):
        for j in range(attributeTrainReduced.shape[1]):
            if attributeTrainReduced.iloc[i,j]=="None" or attributeTrainReduced.iloc[i,j]=="none":
                attributeTrainReduced.iloc[i,j]=np.nan

    

    for i in range(attributeTrainReduced.shape[1]):
        for j in range(attributeTrainReduced.shape[0]):
            if pd.isna(attributeTrainReduced.iloc[j,i]):
                continue
            if attributeTrainReduced.iloc[j,i] not in ['\'','u']:
                continue
            try:
                attributeTrainReduced.iloc[j,i]=re.sub('u\'(.*)\'','\\1',attributeTrainReduced.iloc[j,i])
                attributeTrainReduced.iloc[j,i]=re.sub('\'(.*)\'','\\1',attributeTrainReduced.iloc[j,i])
            except:
                break
    # give zhengdong
    attributeTrainReduced.to_csv(label+".csv")
    attributeTrainReduced=attributeTrainReduced.drop("business_id",axis=1)
###############################################################################################################################
    ### try xgboost 

    import xgboost
    from sklearn.model_selection import train_test_split
    from sklearn import model_selection
    from sklearn.metrics import accuracy_score
    from sklearn.preprocessing import LabelEncoder
    X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(attributeTrainReduced.iloc[:,:-1]),attributeTrainReduced.iloc[:,-1], random_state=0)
    colLabels=pd.get_dummies(attributeTrainReduced.iloc[:,:-1]).columns

    # encode string class values as integers
    %matplotlib inline
    seed = 7

    # fit model no training data
    model = xgboost.XGBRegressor()
    model.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_test, y_test)],eval_metric='rmse',verbose=False)
    #print(model)
    # make predictions for test data
    y_pred = model.predict(X_test)
    #print(model.feature_importances_)


    nameImportancePair=[(x,y) for x,y in zip(colLabels,model.feature_importances_)]
    nameImportancePair=sorted(nameImportancePair,key=lambda x:x[1],reverse=True)


    nameImportancePair

    import re
    def get_important_feature(x,y):
        """
        x is a dummy feature, y is the colnames list
        return the index of columns with that feature
        """
        originFeature=re.sub("()_.*","\\1",x)

        index=[True if re.match(originFeature,x) else False for x in y]

        return index


    ### Do kruskal test on feature

    from  scipy.stats import kruskal
    def difference_test(data):
        """
        the input data is like this format: the last column is the average stars,
        other columns are attribute levels, this function will test if the attribute
        levels are significant factors. When we have two levels, we t test, else use 
        anova
        """
        # kruskal test

        alpha=0.1
        res=[]
        for i in range(0,data.shape[1]-1):
            res.append(data[data.iloc[:,i]==1].iloc[:,-1].values)
        if len(res)<2:
            return False
        result,pValue=kruskal(*res)
        if pValue<alpha:
            largestStar=list(map(np.mean,res))
            starIndex=sorted(range(len(largestStar)), key=lambda k: largestStar[k])

            level=[data.columns[x] for x in starIndex[::-1]]
            return (True,level)
        else:
            return False



    dummyBusiness=pd.get_dummies(attributeTrainReduced.iloc[:,:-1])

    maxFeature=40

    testedFeature=[]

    for i in range(min(maxFeature,len(nameImportancePair))):

        if re.sub("()_.*","\\1",nameImportancePair[i][0]) in testedFeature:
            continue


        testedFeature.append(re.sub("()_.*","\\1",nameImportancePair[i][0]))

        dummyBusinessTemp=dummyBusiness.iloc[:,get_important_feature(nameImportancePair[i][0],colLabels)]


        dummyBusinessTemp['stars']=attributeTrainReduced.iloc[:,-1]

        print(difference_test(dummyBusinessTemp))
    print("------------------------------------------------------------")

C:\Users\ykhra\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ykhra\Anaconda3\lib\site-packages\ipykernel_launcher.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, ['WheelchairAccessible_False', 'WheelchairAccessible_True'])
(True, ['Caters_True', 'Caters_False'])
(True, ['latenight_False', 'latenight_True'])
(True, ['street_True', 'street_False'])
(True, ['DriveThru_False', 'DriveThru_True'])
(True, ['RestaurantsReservations_True', 'RestaurantsReservations_False'])
(True, ['lot_True', 'lot_False'])
False
(True, ['validated_True', 'validated_False'])
(True, ['GoodForKids_False', 'GoodForKids_True'])
(True, ['BusinessAcceptsCreditCards_False', 'BusinessAcceptsCreditCards_True'])
(True, ["RestaurantsAttire_'dressy'", "RestaurantsAttire_u'dressy'", "RestaurantsAttire_'casual'", "RestaurantsAttire_u'casual'", "RestaurantsAttire_'formal'", "RestaurantsAttire_u'formal'"])
(True, ['BikeParking_True', 'BikeParking_False'])
(True, ['garage_True', 'garage_False'])
(True, ['trendy_True', 'trendy_False'])
(True, ["NoiseLevel_u'quiet'", "NoiseLevel_'quiet'", "NoiseLevel_u'average'", "NoiseLevel_'average'", "NoiseLevel_u'loud'", "NoiseLevel_'loud'", "No

C:\Users\ykhra\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ykhra\Anaconda3\lib\site-packages\ipykernel_launcher.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, ['WheelchairAccessible_False', 'WheelchairAccessible_True'])
(True, ['street_True', 'street_False'])
(True, ['dessert_False', 'dessert_True'])
False
(True, ['latenight_False', 'latenight_True'])
(True, ['Caters_True', 'Caters_False'])
(True, ['classy_True', 'classy_False'])
(True, ['GoodForKids_False', 'GoodForKids_True'])
False
(True, ['BikeParking_True', 'BikeParking_False'])
(True, ["NoiseLevel_'quiet'", "NoiseLevel_u'quiet'", "NoiseLevel_'average'", "NoiseLevel_u'average'", "NoiseLevel_'loud'", "NoiseLevel_u'loud'", "NoiseLevel_u'very_loud'", "NoiseLevel_'very_loud'"])
(True, ['DriveThru_False', 'DriveThru_True'])
(True, ['trendy_True', 'trendy_False'])
(True, ['HasTV_False', 'HasTV_True'])
(True, ["Alcohol_'beer_and_wine'", "Alcohol_u'beer_and_wine'", "Alcohol_'full_bar'", "Alcohol_u'full_bar'", "Alcohol_u'none'", "Alcohol_'none'"])
(True, ['dinner_True', 'dinner_False'])
(True, ["RestaurantsAttire_'dressy'", "RestaurantsAttire_u'dressy'", "RestaurantsAttire_'casual'", "Res

C:\Users\ykhra\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ykhra\Anaconda3\lib\site-packages\ipykernel_launcher.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, ['WheelchairAccessible_False', 'WheelchairAccessible_True'])
(True, ['Caters_True', 'Caters_False'])
(True, ['latenight_False', 'latenight_True'])
(True, ["RestaurantsAttire_'dressy'", "RestaurantsAttire_'casual'", "RestaurantsAttire_u'dressy'", "RestaurantsAttire_u'formal'", "RestaurantsAttire_u'casual'", "RestaurantsAttire_'formal'"])
(True, ['dinner_True', 'dinner_False'])
(True, ['RestaurantsTableService_False', 'RestaurantsTableService_True'])
(True, ['BusinessAcceptsBitcoin_True', 'BusinessAcceptsBitcoin_False'])
(True, ['RestaurantsGoodForGroups_False', 'RestaurantsGoodForGroups_True'])
(True, ['HasTV_False', 'HasTV_True'])
(True, ["NoiseLevel_'quiet'", "NoiseLevel_'average'", "NoiseLevel_u'quiet'", "NoiseLevel_u'average'", "NoiseLevel_'loud'", "NoiseLevel_u'loud'", "NoiseLevel_u'very_loud'", "NoiseLevel_'very_loud'"])
(True, ['BusinessAcceptsCreditCards_False', 'BusinessAcceptsCreditCards_True'])
(True, ['lunch_True', 'lunch_False'])
(True, ['RestaurantsPriceRange2_4', '

C:\Users\ykhra\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\ykhra\Anaconda3\lib\site-packages\ipykernel_launcher.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, ['dinner_True', 'dinner_False'])
(True, ['casual_True', 'casual_False'])
(True, ['Caters_True', 'Caters_False'])
(True, ['RestaurantsTableService_True', 'RestaurantsTableService_False'])
(True, ['breakfast_False', 'breakfast_True'])
(True, ['RestaurantsReservations_True', 'RestaurantsReservations_False'])
(True, ['DriveThru_False', 'DriveThru_True'])
False
(True, ['validated_True', 'validated_False'])
(True, ['street_True', 'street_False'])
False
(True, ["WiFi_u'free'", "WiFi_u'no'", "WiFi_'no'", "WiFi_'paid'", "WiFi_u'paid'", "WiFi_'free'"])
False
(True, ['lot_True', 'lot_False'])
(True, ['RestaurantsDelivery_True', 'RestaurantsDelivery_False'])
(True, ['BikeParking_True', 'BikeParking_False'])
(True, ['divey_True', 'divey_False'])
(True, ['OutdoorSeating_True', 'OutdoorSeating_False'])
(True, ['lunch_True', 'lunch_False'])
(True, ['BusinessAcceptsCreditCards_False', 'BusinessAcceptsCreditCards_True'])
(True, ['RestaurantsPriceRange2_4', 'RestaurantsPriceRange2_3', 'Restaurants